In [1]:
import tensorflow as tf

import scipy.io.wavfile as wav
from tensorflow.contrib import rnn
import numpy as np
import os

unhappy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/0/'
unhappy_directory = os.fsencode(unhappy_directory_path)

happy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/1/'
happy_directory = os.fsencode(happy_directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt"

In [2]:
import sys

sys.path.insert(0, '/media/saurabh/New Volume/tf_audio_sentiment/pyAudioAnalysis')

from  pyAudioAnalysis  import audioBasicIO
from  pyAudioAnalysis import audioFeatureExtraction

In [3]:
def audio_to_features(fileurl):
    
    [Fs, x] = audioBasicIO.readAudioFile(fileurl);
    F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
    
    a=F[:21,:]
    b=F[33]

    b=np.expand_dims(b, axis=0)

    #print(b.shape)


    d = np.concatenate( (a, b), axis=0)
    #print(d.shape)
    d = np.swapaxes(d, 0, 1)
    return d

In [4]:
timesteps=2000 # max timesteps
num_input = 22 # size of vector, at each timestep
num_classes = 2 # happy or sad

num_hidden = 110 # hidden layer num of features

x = tf.placeholder(tf.float32, [None,timesteps, num_input]) 
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy



In [5]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]
        

In [6]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [7]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
        
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [8]:
Logits = BiRNN(x, weights, biases)
#prediction = tf.nn.softmax(Logits) # use only for inference, softmax included in loss function

In [9]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()



In [10]:


with tf.Session() as sess:
    # Initialize variables
    sess.run(init)

    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path)
    print("Model restored from file: %s" % load_path)
    
    accoustic_features = pad(audio_to_features('/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/1/16_19_24015.wav'))
    
    
    res=sess.run([Logits], feed_dict={x: [accoustic_features]}) 
    
    print(res)

INFO:tensorflow:Restoring parameters from /media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt


INFO:tensorflow:Restoring parameters from /media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt


Model restored from file: None
[array([[ 2.842103  , -2.97330403]], dtype=float32)]


In [ ]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=1)

In [ ]:
res

In [ ]:
with tf.Session() as sess2:
    
    print(sess2.run([soft]))